In [ ]:
import marimo as mo

In [ ]:
import random
import pyscipopt as scip
from ortools.sat.python import cp_model

# 1 機械リリース時刻付き重み付き完了時刻和最小化問題

- 機械: 1 つだけ
- ジョブ $J = \{ 1, \dots, n \}$
- 各ジョブの処理時間: $p_j \ (\forall j \in J)$
- 各ジョブの重要度: $w_j \ (\forall j \in J)$
- 各ジョブのリリース時刻: $r_j \ (\forall j \in J)$
- 各ジョブの処理完了時刻: $C_j \ (\forall j \in J)$

$C_j$ の重み付き和を最小化する.

## 離接定式化(Disjunctive formulation)

$M$ を大きな定数として

\begin{align*}
&\text{minimize} &\sum_{j=1}^n w_j s_j + \sum_{j=1}^n w_j p_j \\
&\text{s.t.} &s_j + p_j - M (1 - x_{jk}) &\leq s_k \ &(\forall j \neq k) \\
& &x_{jk} + x_{kj} &= 1 \ &(\forall j < k) \\
& &s_j &\geq r_j \ &(\forall j \in J) \\
& &x_{jk} &\in \{0, 1\} \ &(\forall j \neq k)
\end{align*}

- 決定変数
    - $s_j$: ジョブ $j$ の開始時刻
    - $x_{jk}$: ジョブ $j$ がジョブ $k$ に先行するとき $1$
- 補足
    - 目的関数の第 2 項目は定数であるため第 1 項だけを最小化すればよい

## 実装

In [ ]:
def make_data(n):
    """
    Data generator for the one machine scheduling problem.
    """

    random.seed(0)
    p, r, d, w = {}, {}, {}, {}

    J = range(1, n + 1)

    for j in J:
        p[j] = random.randint(1, 4)
        w[j] = random.randint(1, 3)

    T = sum(p)
    for j in J:
        r[j] = random.randint(0, 5)
        d[j] = r[j] + random.randint(0, 5)

    return J, p, r, d, w

In [ ]:
class ModelDisjunctiveScip:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.scip = scip.Model("scheduling: disjunctive")

        # Big M
        M = max(self.r.values()) + sum(self.p.values())

        # start time variable, x[j,k] = 1 if job j precedes job k, 0 otherwise
        self.x = {}
        self.s = {}

        ub = sum(self.p[j] for j in self.J)
        for j in self.J:
            self.s[j] = self.scip.addVar(lb=r[j], ub=ub, vtype="C", name=f"s[{j}]")
            for k in self.J:
                if j != k:
                    self.x[j, k] = self.scip.addVar(vtype="B", name=f"x[{j},{k}]")

        for j in self.J:
            for k in self.J:
                if j != k:
                    self.scip.addCons(
                        self.s[j] - self.s[k] + M * self.x[j, k] <= (M - self.p[j]), f"Bound[{j},{k}]"
                    )

                if j < k:
                    self.scip.addCons(self.x[j, k] + self.x[k, j] == 1, f"Disjunctive[{j},{k}]")

        self.scip.setObjective(scip.quicksum(self.w[_j] * self.s[_j] for _j in self.J), sense="minimize")

    def solve(self) -> None:
        self.scip.optimize()

    def get_z(self):
        return self.scip.getObjVal() + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.scip.getVal(self.s[_j]) + 0.5), _j) for _j in self.s])]

In [ ]:
class _MyInterval:
    # [start, end)
    def __init__(self, scip: scip.Model, lb: int, ub: int, size: int):
        self.lb = lb
        self.ub = ub
        self.size = size
        self.start = scip.addVar(lb=self.lb, ub=self.ub - self.size, vtype='C')
        self.end = self.start + self.size

def _my_add_no_overlap(scip: scip.Model, jobs: dict[int, _MyInterval]):
    if len(jobs) == 0:
        return
    for _j1 in jobs.keys():
        for _j2 in jobs.keys():
            if _j2 <= _j1:
                continue
            big_m = max(jobs[_j1].ub - jobs[_j2].lb, jobs[_j2].ub - jobs[_j1].lb)
            tmp1 = scip.addVar(vtype='B')
            scip.addCons(
                jobs[_j1].end <= jobs[_j2].start + big_m * (1 - tmp1)
            )
            tmp2 = scip.addVar(vtype='B')
            scip.addCons(
                jobs[_j2].end <= jobs[_j1].start + big_m * (1 - tmp2)
            )
            scip.addCons(tmp1 + tmp2 >= 1)

class ModelIntervalScip:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.scip = scip.Model("scheduling: disjunctive")

        ub = sum(p[_j] for _j in J)
        self.jobs = {_j: _MyInterval(self.scip, self.r[_j], ub, self.p[_j]) for _j in self.J}
        _my_add_no_overlap(self.scip, self.jobs)

        self.scip.setObjective(scip.quicksum(self.w[_j] * self.jobs[_j].start for _j in self.J), sense="minimize")

    def solve(self) -> None:
            self.scip.optimize()

    def get_z(self):
        return self.scip.getObjVal() + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.scip.getVal(self.jobs[_j].start) + 0.5), _j) for _j in self.jobs])]

In [ ]:
class ModelDisjunctiveCpSat:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.cp = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        # Big M
        M = max(r.values()) + sum(p.values())

        # start time variable, x[j,k] = 1 if job j precedes job k, 0 otherwise
        self.x = {}
        self.s = {}

        ub = sum(p[j] for j in J)
        for j in J:
            self.s[j] = self.cp.new_int_var(lb=r[j], ub=ub, name=f"s[{j}]")
            for k in J:
                if j != k:
                    self.x[j, k] = self.cp.new_bool_var(name=f"x[{j},{k}]")

        for j in J:
            for k in J:
                if j != k:
                    self.cp.add(self.s[j] - self.s[k] + M * self.x[j, k] <= (M - p[j]))

                if j < k:
                    self.cp.add(self.x[j, k] + self.x[k, j] == 1)

        self.cp.minimize(sum(self.w[j] * self.s[j] for j in J))

    def solve(self) -> None:
        self.solver.parameters.log_search_progress = True
        self.solver.solve(self.cp)

    def get_z(self):
        return self.solver.objective_value + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(int(self.solver.value(self.s[_j]) + 0.5), _j) for _j in self.s])]

In [ ]:
class ModelIntervalCpSat:
    def __init__(self, J, p, r, w):
        self.J = J
        self.p = p
        self.r = r
        self.w = w

        self.cp = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        ub = sum(p[_j] for _j in J)
        s = {_j: self.cp.new_int_var(lb=self.r[_j], ub=ub, name=f"s[{_j}]") for _j in self.J}
        self.jobs = {_j: self.cp.new_fixed_size_interval_var(s[_j], self.p[_j], f"jobs[{_j}]") for _j in self.J}
        self.cp.add_no_overlap(self.jobs.values())

        self.cp.minimize(sum(self.w[_j] * self.jobs[_j].start_expr() for _j in J))

    def solve(self) -> None:
        self.solver.parameters.log_search_progress = True
        self.solver.solve(self.cp)

    def get_z(self):
        return self.solver.objective_value + sum([self.w[_j] * self.p[_j] for _j in self.J])

    def get_seq(self):
        return [_j for (_t, _j) in sorted([(self.solver.value(self.jobs[_j].start_expr()), _j) for _j in self.jobs])]

In [ ]:
n = 30
J, p, r, d, w = make_data(n)

In [ ]:
run_scip_disj = mo.ui.run_button(label="Run", full_width=True)
run_scip_disj

In [ ]:
if run_scip_disj.value:
    _model = ModelDisjunctiveScip(J, p, r, w)
    with mo.redirect_stderr():
        _model.solve()

    _z = _model.get_z()
    _seq = _model.get_seq()
    mo.md(f"""
    - Opt.value by Disjunctive Formulation: {_z}
    - Solution: {_seq}
    """)

In [ ]:
run_scip_interval = mo.ui.run_button(label="Run", full_width=True)
run_scip_interval

In [ ]:
if run_scip_interval.value:
    _model = ModelIntervalScip(J, p, r, w)
    with mo.redirect_stderr():
        _model.solve()

    _z = _model.get_z()
    _seq = _model.get_seq()
    mo.md(f"""
    - Optimal value: {_z}
    - Solution: {_seq}
    """)

In [ ]:
_model = ModelDisjunctiveCpSat(J, p, r, w)
with mo.redirect_stderr():
    _model.solve()

_z = _model.get_z()
_seq = _model.get_seq()
mo.md(f"""
- Opt.value by Disjunctive Formulation: {_z}
- Solution: {_seq}
""")


Starting CP-SAT solver v9.13.4784
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x89d2d6e0295b344b)
#Variables: 900 (#ints: 30 in objective) (464 primary variables)
  - 870 Booleans in [0,1]
  - 5 in [0,74]
  - 4 in [1,74]
  - 9 in [2,74]
  - 4 in [3,74]
  - 2 in [4,74]
  - 6 in [5,74]
#kLinear2: 435
#kLinear3: 870

Starting presolve at 0.00s
  1.44e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.16e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  9.48e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  4.27e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.36e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'234 nodes and 3'074 arcs.
[Symmetry] Symmetry computation done. time: 0.000337891 dtime: 0.00044344
  1.40e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.54e-03s  5.07e-04d  [Probe] #probed=1'740 
  3.43e-06s  0.00e+00d

  2.39e-04s  0.00e+00d  [DetectDifferentVariables] #different=435 #cliques=1 #size=30 
  5.58e-06s  0.00e+00d  [ProcessSetPPC] 
  5.88e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.45e-05s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  1.30e-04s  6.99e-05d  [FindBigVerticalLinearOverlap] 
  4.41e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  2.83e-06s  0.00e+00d  [MergeClauses] 
  1.40e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.08e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.39e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.05e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.59e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.39e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'265 nodes and 3'134 arcs.
[Symmetry] Symmetry computation done. time: 0.000248331 dtime: 0.00044797
  1.56e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
 

  1.43e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.07e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.02e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.37e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.21e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.31e-06s  0.00e+00d  [DetectDominatedLinearConstraints] 
  1.19e-04s  0.00e+00d  [DetectDifferentVariables] 
  4.32e-06s  0.00e+00d  [ProcessSetPPC] 
  5.19e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.34e-05s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  1.13e-04s  6.99e-05d  [FindBigVerticalLinearOverlap] 
  3.88e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  2.49e-06s  0.00e+00d  [MergeClauses] 
  1.36e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.07e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.28e-04s  0.00e+00d  [ExpandObjective] 

Presolve summary:
  - 435 affine relations were detected.
  - rul

#1       0.04s best:1650  next:[157,1649] fixed
#2       0.05s best:1618  next:[157,1617] fixed
#3       0.05s best:1615  next:[157,1614] fixed


#Bound   0.06s best:1615  next:[159,1614] quick_restart_no_lp
#Bound   0.06s best:1615  next:[1208,1614] max_lp
#4       0.06s best:1609  next:[1208,1608] quick_restart_no_lp


...

#Bound  19.02s best:1472  next:[1470,1471] reduced_costs


#Bound  19.07s best:1472  next:[1471,1471] reduced_costs


#Model  19.14s var:425/465 constraints:834/901


#Done   19.23s reduced_costs
#Done   19.23s core



Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                        'core':         1 [  19.21s,   19.21s]   19.21s   0.00ns   19.21s         1 [  14.32s,   14.32s]   14.32s   0.00ns   14.32s
                  'default_lp':         1 [  19.21s,   19.21s]   19.21s   0.00ns   19.21s         1 [   5.41s,    5.41s]    5.41s   0.00ns    5.41s
            'feasibility_pump':         1 [  4.49ms,   4.49ms]   4.49ms   0.00ns   4.49ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                       'fixed':         1 [  19.21s,   19.21s]   19.21s   0.00ns   19.21s         1 [  12.48s,   12.48s]   12.48s   0.00ns   12.48s
                          'fj':         1 [ 48.93ms,  48.93ms]  48.93ms   0.00ns  48.93ms         1 [100.25ms, 100.25ms] 100.25ms   0.00ns 100.25ms
                          'fj':         1 [ 66.17ms,  66.17ms]  66.17ms   0.00ns  66.17ms         1 [ 95.29ms, 

In [ ]:
_model = ModelIntervalCpSat(J, p, r, w)
with mo.redirect_stderr():
    _model.solve()

_z = _model.get_z()
_seq = _model.get_seq()
mo.md(f"""
- Optimal value: {_z}
- Solution: {_seq}
""")


Starting CP-SAT solver v9.13.4784
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x77d9b8aa7f03b1b1)
#Variables: 30 (#ints: 30 in objective) (30 primary variables)
  - 5 in [0,74]
  - 4 in [1,74]
  - 9 in [2,74]
  - 4 in [3,74]
  - 2 in [4,74]
  - 6 in [5,74]
#kInterval: 30
#kNoOverlap: 1 (#intervals: 30)

Starting presolve at 0.00s
  1.43e-05s  0.00e+00d  [DetectDominanceRelations] 
  5.40e-05s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.23e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.70e-06s  0.00e+00d  [DetectDuplicateColumns] 
  1.18e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 61 nodes and 60 arcs.
[Symmetry] Symmetry computation done. time: 2.3064e-05 dtime: 8.17e-06
[Symmetry] #generators: 2, average support size: 2
[Symmetry] 2 orbits on 4 variables with sizes: 2,2
[Symmetry] Found orbitope of size 1 x 2
  1.09e-04s  0.00e+00d  [D

#4       0.01s best:1596  next:[1409,1595] quick_restart
#Bound   0.01s best:1596  next:[1410,1595] max_lp_sym
#Bound   0.02s best:1596  next:[1418,1595] max_lp_sym
#5       0.02s best:1588  next:[1418,1587] quick_restart_no_lp


#6       0.03s best:1584  next:[1418,1583] quick_restart
#7       0.03s best:1583  next:[1418,1582] quick_restart_no_lp


#8       0.05s best:1582  next:[1418,1581] quick_restart_no_lp
#9       0.05s best:1579  next:[1418,1578] quick_restart
#10      0.05s best:1575  next:[1418,1574] quick_restart_no_lp
#11      0.05s best:1569  next:[1418,1568] quick_restart


#12      0.06s best:1565  next:[1418,1564] quick_restart_no_lp
#13      0.06s best:1564  next:[1418,1563] quick_restart_no_lp


...

#Bound   1.90s best:1472  next:[1443,1471] max_lp_sym


#Bound   2.20s best:1472  next:[1444,1471] max_lp_sym
#Bound   2.20s best:1472  next:[1445,1471] max_lp_sym


#Bound   2.22s best:1472  next:[1449,1471] max_lp_sym


#Done    2.43s max_lp_sym


#Bound   2.43s best:1472  next:[1471,1471] max_lp_sym [skipped_logs=5]

Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                        'core':         1 [   2.43s,    2.43s]    2.43s   0.00ns    2.43s         1 [   2.66s,    2.66s]    2.66s   0.00ns    2.66s
                  'default_lp':         1 [   2.43s,    2.43s]    2.43s   0.00ns    2.43s         1 [   1.17s,    1.17s]    1.17s   0.00ns    1.17s
            'feasibility_pump':         1 [ 79.03us,  79.03us]  79.03us   0.00ns  79.03us         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                       'fixed':         1 [   2.43s,    2.43s]    2.43s   0.00ns    2.43s         1 [   2.25s,    2.25s]    2.25s   0.00ns    2.25s
                          'fj':         1 [ 41.21ms,  41.21ms]  41.21ms   0.00ns  41.21ms         1 [100.07ms, 100.07ms] 100.07ms   0.00ns 100.07ms
                          'fj':         